In [54]:
import pandas as pd
import datetime
from sklearn.neural_network import MLPClassifier
import random

In [55]:
seq_len = 14
selected_user = 1032
user_lst = [290, 407, 581, 1032, 1436]
def match_dataset_date(selected_user, seq_len):
    y_df = pd.read_csv(f'/mnt/results/user_{selected_user}_puqe.csv')
    x_df = pd.read_csv(f'/mnt/results/user_{selected_user}_activity_bodyport_hyperimpute_with_date.csv')
    y_df['date'] = pd.to_datetime(y_df['date'])
    x_df['date'] = pd.to_datetime(x_df['date'])
    dataset = []
    for i in range(len(y_df)):
        row = y_df.iloc[i]
        day_before = row['date'] - datetime.timedelta(days=seq_len)
        x = (x_df[(x_df['date'] >= day_before) & (x_df['date'] <= row['date'])])
        dataset.append((x.drop('date', axis=1).to_numpy(), row['answer_text']))
    return dataset
# gnn_dataset_1032 = match_dataset_date(selected_user, seq_len)
# gnn_dataset_1032 = list(filter(lambda x: len(x[0])>=seq_len, gnn_dataset))
# baseline_dataset_1032 = list(map(lambda x: (x[0][-7:], x[1]), gnn_dataset))
baseline_ds_lst = []
for u in user_lst:
    gnn_dataset = match_dataset_date(u, seq_len)
    gnn_dataset = list(filter(lambda x: len(x[0])>=seq_len, gnn_dataset))
    baseline_dataset = list(map(lambda x: (x[0][-7:], x[1]), gnn_dataset))
    baseline_ds_lst.append(baseline_dataset)


In [56]:
baseline_ds = []
for ds in baseline_ds_lst:
    baseline_ds.extend(ds)
len(baseline_ds)
random.seed(90)
random.shuffle(baseline_ds)

In [57]:
baseline_x = list(map(lambda x: x[0].ravel(), baseline_ds))
baseline_y = list(map(lambda x: x[1], baseline_ds))
baseline_x_train = baseline_x[:120]
baseline_x_test = baseline_x[120:]
baseline_y_train = baseline_y[:120]
baseline_y_test = baseline_y[120:]

In [58]:
clf = MLPClassifier(random_state=90, max_iter=300).fit(baseline_x_train, baseline_y_train)
clf.score(baseline_x_test, baseline_y_test)

0.5